# Pelatihan Model Klasifikasi Gambar Bahasa Isyarat

## Pengumpulan Dataset

Unduh dataset [Sign Language MNIST](https://www.kaggle.com/datasets/datamunge/sign-language-mnist) dari laman [Kaggle](https://www.kaggle.com) dan ekstrak file terkompresinya.

## Muat Pakcage

Setelah mengunduh dataset, import semua package yang dibutuhkan untuk memproses data dan membuat model.

In [ ]:
import tensorflow
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd
import numpy as np
import os
import cv2

## Muat Dataset

Dataset yang akan digunakan adalah [Dataset Sistem Isyarat Bahasa Indonesia](https://www.kaggle.com/datasets/alvinbintang/sibi-dataset). Unduh dataset terlebih dahulu dan taruh folder dataset di dalam folder yang sama dengan python notebook ini. Kemudian masukkan nama folder dataset ke dalam variabel `training_data_directory`.

In [ ]:
training_data_directory = 'SIBI'


# Helper function to load images from given directories
def load_images(directory, uniq_labels):
    images = []
    labels = []
    for idx, label in enumerate(uniq_labels):
        for file in os.listdir(directory + "/" + label):
            filepath = directory + "/" + label + "/" + file
            image = cv2.resize(
                cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2GRAY), (28, 28))
            images.append(image)
            labels.append(idx)
    images = np.array(images)
    labels = np.array(labels)
    return (images, labels)

In [ ]:
uniq_labels = sorted(os.listdir(training_data_directory))
images, labels = load_images(
    directory=training_data_directory, uniq_labels=uniq_labels)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, stratify=labels)

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

n = len(uniq_labels)
train_n = len(X_train)
test_n = len(X_test)

print("Total Jumlah Huruf: ", n)
print("Jumlah data gambar latih: ", train_n)
print("Jumlah data gambar tes: ", test_n)

In [ ]:
uniq_labels

Setelah memuat semua package yang dibutuhkan, maka buat variabel untuk memuat dataset.

Label dengan bilangan positif yang mewakili tiap huruf (0-25 : A-Z) akan diubah menjadi larik/array.
misal huruf "A" yang dilabeli dengan angka "0" akan diubah menjadi larik/array 

"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"

 dan huruf "B" menjadi larik/array
 
 "[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]" dan seterusnya sampai huruf terakhir.
 
 Proses ini disebut binerisasi label yang menghasilkan label dalam bentuk larik/array angka biner yang disebut "One-Hot".

In [ ]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [ ]:
print(y_train[0])
print(len(y_train[0]))

Beralih ke data latih dan data tes yang sudah dipisahkan dengan label, data tersebut berisi bilangan positif mulai dari 0-255 untuk tiap pixel gambar. Angka tersebut akan diubah dari rentang angka 0-255 menjadi 0-1 supaya mempermudah perhitungan dalam model. Proses ini dinamakan "Normalization" atau Normalisasi data inputan.

In [ ]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

Dalam melatih model, terutama yang akan digunakan adalah [pembelajaran mendalam / deep learning](https://www.dicoding.com/blog/mengenal-deep-learning/), prinsip yang bekerja adalah semakin banyak jumlah dan variasi data yang dilatih, maka semakin baik pula hasil prediksi dari sebuah model. Oleh karena itu, dataset saat ini akan diperbanyak/[augmentasi](https://blog.algorit.ma/augmentasi-data/) dengan rincian sebagai berikut.
1. Secara random menambah variasi rotasi gambar/data dengan perubahan 10 derajat. [rotation_range=10]
2. Secara random menambah variasi zoom gambar/data dengan perubahan 10%. [zoom_range=0.1]
3. Secara random menambah variasi pergeseran gambar dari titik tengah secara horisontal sebesar 10% dari lebar gambar. [width_shift_range=0.1]
4. Secara random menambah variasi pergeseran gambar dari titik tengah secara vertikal sebesar 10% dari tinggi gambar. [height_shift_range=0.1]

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    # randomly rotate images in the range (degrees, 0 to 180)
    rotation_range=10,
    zoom_range=0.1,  # Randomly zoom image
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

## Perancangan Model

Model yang akan digunakan adalah model deep learning dengan arsitektur [Convolutional Neural Network](https://medium.com/@16611110/apa-itu-convolutional-neural-network-836f70b193a4) atau bisa disingkat [CNN](https://medium.com/@16611110/apa-itu-convolutional-neural-network-836f70b193a4).

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

model = keras.models.Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding='same',
          activation='relu', input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding='same'))
model.add(Conv2D(50, (3, 3), strides=1, padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding='same'))
model.add(Conv2D(25, (3, 3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding='same'))
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=24, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

## Pelatihan Model

Kemudian melatih model dengan code dibawha

In [ ]:
history = model.fit(datagen.flow(X_train, y_train, batch_size = 128) ,epochs = 20 , validation_data = (x_test, y_test) , callbacks = [learning_rate_reduction])

## Evaluasi Model

Setelah melatih model, model akan dievaluasi dengan menggunakan data tes untuk melihat seberapa akurat dalam memprediksi huruf berdasarkan pada data input.

In [ ]:
model.evaluate(X_test, y_test)

Kemudian, untuk dapat melihat perkembangan metrik model pada saat pelatihan dapat dilakukan dengan membuat grafik pelatihan.

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

Selanjutnya menyimpan model yang sudah dilatih ke dalam sebuah file untuk digunakan di dalam aplikasi. Model disimpan dengan format keras model dengan ekstensi file .h5

In [ ]:
model.save('smnist.h5')

referensi : https://www.kaggle.com/code/muhammadibrahimqasmi/sign-language-mnist-cnn-simple-solution